# 2022 Flatiron Machine Learning x Science Summer School

## Step 8: Run symbolic regression on bottleneck DSN predictions

In this step, we perform symbolic regression on the latent feature and target predictions of bottleneck DSNs.

In [272]:
import os
import numpy as np
import joblib
from pysr import PySRRegressor
import sympy
from scipy import optimize

import warnings
warnings.filterwarnings("ignore")

### Step 8.1: Load data

We load the input data as well as the latent feature and target prediction data.

In [2]:
data_path = "data_1k"
data_ext = ".gz"

In [3]:
data = {}
for file_name in os.listdir(data_path):
    if file_name[-len(data_ext):] == data_ext:
        
        var = file_name[:-len(data_ext)]
        var_data = np.loadtxt(os.path.join(data_path, file_name))
        if len(var_data.shape) == 1:
            var_data = var_data.reshape(-1,1)
        data[var] = var_data

        print(f"Loaded {var} data.")

Loaded F00 data.
Loaded F00_p1 data.
Loaded F01 data.
Loaded F02 data.
Loaded F03 data.
Loaded F04 data.
Loaded F05 data.
Loaded F06 data.
Loaded F07 data.
Loaded F07_p1 data.
Loaded F08 data.
Loaded G00 data.
Loaded G00_p1 data.
Loaded G01 data.
Loaded G02 data.
Loaded G03 data.
Loaded G04 data.
Loaded G05 data.
Loaded G06 data.
Loaded G07 data.
Loaded G07_p1 data.
Loaded G08 data.
Loaded X00 data.
Loaded X01 data.
Loaded X02 data.
Loaded X03 data.
Loaded X04 data.
Loaded X05 data.
Loaded X06 data.
Loaded X07 data.
Loaded X08 data.


In [4]:
mask_ext = ".mask"

masks = {}
for var in data:
    if var[0] == "X":
        masks[var] = joblib.load(os.path.join(data_path, var + mask_ext))
        print(f"Loaded masks for {var} data.")

Loaded masks for X00 data.
Loaded masks for X01 data.
Loaded masks for X02 data.
Loaded masks for X03 data.
Loaded masks for X04 data.
Loaded masks for X05 data.
Loaded masks for X06 data.
Loaded masks for X07 data.
Loaded masks for X08 data.


### Step 8.2: Run PySR

In [5]:
def get_model(X, y):

    model = PySRRegressor(
        procs=4,
        populations=30,
        niterations=30,
        maxsize=20,
        binary_operators=["plus", "sub", "mult"],
        unary_operators=["sin", "cos", "exp", "log_abs"],      
        model_selection="best",
        verbosity=0
    )

    model.fit(X, y)

    return model

In [6]:
model_path = "models"
model_name = "pysr_models_1k_preds.pkl"

try:
    models = joblib.load(os.path.join(model_path, model_name))
except:
    models = {}

In [7]:
# vars = sorted([k for k in data.keys() if k[0] == "F" and k not in models])
vars = ["F07_p1"]

for var in vars:

    # get target dimensions
    f_dim = data[var].shape[1]

    # get input variables
    g_var = "G" + var[1:]
    x_var = "X" + var[1:].split('_')[0]

    # get training mask
    mask = masks[x_var]["train"]

    models[var] = {g_var: [], x_var: []}
    for i in range(f_dim):

        # get target data
        y = data[var][mask,i]

        # learn f(x)
        print(f"Learning {var}_{i}({g_var}).")
        X = data[g_var][mask]
        models[var][g_var].append(get_model(X, y))

        joblib.dump(models, os.path.join(model_path, model_name))
    
        # learn f(g(x))
        print(f"Learning {var}_{i}({x_var}).")
        X = data[x_var][mask]
        models[var][x_var].append(get_model(X, y))

        joblib.dump(models, os.path.join(model_path, model_name))

    # get target dimensions
    g_dim = data[g_var].shape[1]

    models[g_var] = {x_var: []}
    for i in range(g_dim):

        # get target data
        y = data[g_var][mask,i]
   
        # learn g(x)
        print(f"Learning {g_var}_{i}({x_var}).")
        X = data[x_var][mask]
        models[g_var][x_var].append(get_model(X, y))

        joblib.dump(models, os.path.join(model_path, model_name))

Learning F07_p1_0(G07_p1).


Learning F07_p1_0(X07).
Learning G07_p1_0(X07).
Learning G07_p1_1(X07).
Learning G07_p1_2(X07).


In [ ]:
del_name = "hall_of_fame_"

for f in os.listdir():
    if del_name in f:
        os.remove(f)
        print(f"Deleted {f}.")

### Step 8.3: Check discovery

In [8]:
disc_eps = 1e-3

In [9]:
for d_var in models:
    for i_var in models[d_var]:
        for m, model in enumerate(models[d_var][i_var]):
            best = model.get_best()
            print(f"{d_var}_{m}({i_var}): {best.loss:.2e} - [{(' ','X')[best.loss < disc_eps]}] - {best.equation}")

F00_p1_0(G00_p1): 2.35e-04 - [X] - ((x2 + x0) + x1)
F00_p1_0(X00): 1.37e-03 - [ ] - (cos(x1) + ((x0 + x1) * x0))
G00_p1_0(X00): 1.91e-04 - [X] - cos(x1)
G00_p1_1(X00): 2.09e-04 - [X] - (x1 * x0)
G00_p1_2(X00): 9.91e-04 - [X] - (x0 * x0)
F07_p1_0(G07_p1): 2.68e-13 - [X] - ((((x2 + (x1 * 0.99203295)) + (x0 * 1.4198973)) * -1.882113) + (x0 + 0.22840188))
F07_p1_0(X07): 2.98e+00 - [ ] - (((x0 * ((x0 + (x1 * 1.730558)) * 2.6368675)) + (log_abs(log_abs(x1)) * 2.402944)) - -1.828079)
G07_p1_0(X07): 7.34e-02 - [ ] - ((x0 * x0) * -1.2697303)
G07_p1_1(X07): 4.61e-02 - [ ] - (0.4249124 - (cos(x1 * -2.9942389) * 1.751644))
G07_p1_2(X07): 9.82e-01 - [ ] - (x0 * (x1 * -2.3588479))


### Step 8.4: Get full model and optimize constants

In [273]:
d_var = "F07_p1"
l_var = "G07_p1"
i_var = "X07"

In [274]:
def num2symbols(expr):
    
    w = sympy.Wild("w", properties=[lambda t: isinstance(t, sympy.Float)])
    n = expr.find(w)
    
    alphabet = [f"p{p}" for p in range(len(n))]
    s = sympy.symbols(" ".join(alphabet[:len(n)]))
    
    d = {k: v for k, v in zip(n, s)}
    
    return expr.subs(d), [float(k) for k in d]

def optimize_eq(eq, inits, in_data, target_data):

    i_syms = [f"x{i}" for i in range(in_data.shape[1])]
    
    def opt_fun(p):
        opt_eq = eq.subs({f"p{i}": p[i] for i in range(len(p))})
        return np.mean((sympy.lambdify(i_syms, opt_eq, modules='numpy')(*list(in_data.T)) - target_data)**2)
    
    res = optimize.minimize(opt_fun, inits, method="BFGS")
    opt_eq = eq.subs({f"p{i}": res.x[i] for i in range(len(res.x))})
        
    return opt_eq

In [275]:
l_eqs = []
l_syms = []

for m, model in enumerate(models[l_var][i_var]):
    l_eqs.append(model.get_best().sympy_format)
    l_syms.append(sympy.Symbol(f'x{m}'))
    
    print(model.get_best().equation)

((x0 * x0) * -1.2697303)
(0.4249124 - (cos(x1 * -2.9942389) * 1.751644))
(x0 * (x1 * -2.3588479))


In [278]:
f_eqs = []

for model in models[d_var][l_var]:
    f_eqs.append(model.get_best().sympy_format.subs(dict(zip(l_syms, l_eqs))))
    
print(f_eqs[0])

2.12350605387058*x0**2 + 4.4396182976127*x0*x1 + 3.27052623751637*cos(2.9942389*x1) - 0.564959757893346


In [279]:
print("2.7*x0**2 + 4.5*x0*x1 + 5.0*cos(3.0*x1)")

2.7*x0**2 + 4.5*x0*x1 + 5.0*cos(3.0*x1)


In [277]:
for f, f_eq in enumerate(f_eqs):
    
    p_eq, inits = num2symbols(f_eq)
    print(p_eq)
    
    t_var = d_var.split('_')[0]
    o_eq = optimize_eq(p_eq, inits, data[i_var], data[t_var][:,f])
    print(o_eq)

p0 + p1*x0**2 + p2*x0*x1 + p3*cos(p4*x1)
2.70000004072343*x0**2 + 4.49999985219734*x0*x1 + 5.00000032796586*cos(2.99999996267622*x1) - 2.07344625560966e-7
